# CREATE DATASETS FOR STORAGE GSP

In [26]:
import numpy as np 
import pandas 
import csv
import warnings
import nltk
import os
import string
import time
import pickle
import seaborn as sns
import random
import csv
from pathlib import Path
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE
warnings.filterwarnings("ignore")
src='/home/shared_data/textmining_genderrecognition_topicextraction/dataset_text_mining_Reuters_RCV2/'

In [30]:
with open(src+'GCP_TEXT_AND_LABEL_PROCESSED.csv', 'r') as temp:
    Data_Set=pandas.read_csv(temp)

In [22]:
train, test_d= train_test_split(Data_Set, test_size=0.30, random_state=42,stratify=Data_Set['code'])
validation, test = train_test_split(test_d, test_size=0.33, random_state=42,stratify=test_d['code'])
train=train.reset_index(drop=True)
validation=validation.reset_index(drop=True)
test=test.reset_index(drop=True)
print('Size iniziale dei record:',len(list(Data_Set['text'])),'\nSize train ds:',len(train),'\nSize validation ds:',len(validation),'\nSize test ds:',len(test))

Size iniziale dei record: 158674 
Size train ds: 111071 
Size validation ds: 31894 
Size test ds: 15709


In [25]:
with open(src+'GCP_TEXT_AND_LABEL_PROCESSED_TRAIN.csv', 'w') as temp:
    train.to_csv(temp, index=False)
with open(src+'GCP_TEXT_AND_LABEL_PROCESSED_VALIDATION.csv', 'w') as temp:
    validation.to_csv(temp, index=False)
with open(src+'GCP_TEXT_AND_LABEL_PROCESSED_TEST.csv', 'w') as temp:
    test.to_csv(temp, index=False)

In [3]:
delete=0
start_row=35
end_row=150 #len(Data_Set)
print('prima--->',len(Data_Set),'!')
delete=len(Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))> end_row) ])+len(Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))< start_row)])
Data_Set = Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))<= end_row) ]
Data_Set = Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))>= start_row) ]
Data_Set = Data_Set.reset_index(drop=True)
print('ora--->',len(Data_Set),', eliminati',delete,'record su',800803 ,'!')

with open("myDictionary_labels.npy", "rb") as tf:
    new_dict = np.load(tf, allow_pickle='TRUE')
new_dict.item()

prima---> 158674 !
ora---> 158674 , eliminati 0 record su 800803 !


In [ ]:
def crea_label_vector(Data_Set):
    y_label_code_reduced=[]
    for index, row in Data_Set.iterrows():
        row=list(row['target'][1:len(row['target'])-1].split(","))
        row_t=[ int(x) for x in row ]
        y_label_code_reduced.append(row_t)
    print('size ridotta',len(y_label_code_reduced))
    return y_label_code_reduced

class_names_list=new_dict.item()
target_names=[x for x in class_names_list.keys()]
y_label_code_reduced=crea_label_vector(Data_Set)

In [ ]:
def stat_ds(y_label_code_reduced):
    x=np.zeros(len(y_label_code_reduced[0]),dtype=int)
    for v in y_label_code_reduced:
        x=x+v
    c=0
    for v in x:
        if v==0:
            c+=1
    print(x)
    print('esempi = 0 :',c)
    #percentuali sull'intero dataset
    xx=x.copy()
    z=[]
    for l in xx:
        z.append(str(round((l/len(Data_Set))*100,1))+'%')
    print(z)
    return x#,[float(x[:4]) for x in z]

def _red_list_bal(y_label_code_reduced):
    c=stat_ds(y_label_code_reduced)
    min_l=min(c)
    red=[0,0,0,0]
    ind=0
    for i in c:
        if i!=min_l:
            val=c[ind]#int((i/100)*len(Data_Set))
            red[ind]=val-min_l#int((min_l/100)*len(Data_Set))
        else:
            red[ind]=1
        ind+=1
    return red

def indx_permutati(Data_Set):
    random.seed(42)
    ind_ds=random.sample(range(len(Data_Set)), len(Data_Set))
    return ind_ds

def bilancia(Data_Set):
    size_prima=len(Data_Set)
    conta1=0
    conta2=0
    conta3=0
    conta4=0
    index=0
    red=_red_list_bal(y_label_code_reduced)
    indx_perm=indx_permutati(Data_Set)
    index_list=[]
    row_list=Data_Set['target'].tolist()
    for row_ind in indx_perm:
        row=row_list[row_ind]
        row=list(row[1:len(row)-1].split(","))
        row_t=[int(x) for x in row]
        if sum(row_t)==1:
            if row_t[0]==1 and conta1<red[0]:
                index_list.append(row_ind)
                conta1+=1
            if row_t[1]==1 and conta1<red[1]:
                index_list.append(row_ind)
                conta2+=1
            if row_t[2]==1 and conta3<red[2]:
                index_list.append(row_ind)
                conta3+=1
            if row_t[3]==1 and conta4<red[3]:
                index_list.append(row_ind)
                conta4+=1
        
        index+=1
        if index%100000==0 and index!=0:
            print('analizzati',index)
    Data_Set.drop(index_list, inplace=True)
    Data_Set.reset_index(drop=True)
    print('eliminati come prima label',conta1,'\neliminati come seconda label',conta2,'\neliminati come terza label',conta3,'\neliminati come quarta label',conta4,'\nsize ridotta da',size_prima,'a',len(Data_Set))
    return Data_Set

stat_ds_list=stat_ds(y_label_code_reduced)  
Data_Set=bilancia(Data_Set)
y_label_code_reduced=crea_label_vector(Data_Set)
stat_ds(y_label_code_reduced)

In [ ]:
# Data_Set=Data_Set.reset_index(drop=True)
Data_Set=Data_Set[['text','code']]
Data_Set

In [7]:
with open(src+'GCP_TEXT_AND_LABEL_PROCESSED.csv', 'w') as temp:
    Data_Set.to_csv(temp, index=False)

# 59 label

In [72]:
with open(src+'GCP_TEXT_AND_LABEL_PROCESSED_59.csv', 'r') as temp:
    Data_Set=pandas.read_csv(temp)

In [73]:
Data_Set

,text,target,code
0,uk share market unlikely make much headway nex...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","MCAT,M11"
1,cbot soft red winter wheat futures closed most...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","MCAT,M14"
2,beijing friday condemned u.s. sanctions impose...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","ECAT,GDIP,E51,GCAT"
3,year march 31 1998 billions yen unless specifi...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","CCAT,C15"
4,pregnant ira bomb suspect released bail friday...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","GCRIM,GVIO,GPOL,GCAT"
...,...,...,...
800798,slovenian development fund state 's restructur...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ...","CCAT,C18"
800799,slow development widely variable corn crop con...,"[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, ...","C21,CCAT,C24"
800800,nbc series quot er quot emmy award sunday best...,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","GENT,GCAT"
800801,sun-diamond growers california went court mond...,"[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","GCRIM,C12,C41"


In [74]:
delete=0
start_row=35
end_row=150 #len(Data_Set)
print('prima--->',len(Data_Set),'!')
delete=len(Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))> end_row) ])+len(Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))< start_row)])
Data_Set = Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))<= end_row) ]
Data_Set = Data_Set[(Data_Set['text'].map(lambda x: len(x.split(' ')))>= start_row) ]
Data_Set = Data_Set.reset_index(drop=True)
print('ora--->',len(Data_Set),', eliminati',delete,'record su',800803 ,'!')

prima---> 800803 !
ora---> 439176 , eliminati 361627 record su 800803 !


In [ ]:
with open("myDictionary_labels_59.npy", "rb") as tf:
    new_dict = np.load(tf, allow_pickle='TRUE')

In [76]:
class_names_list=new_dict.item()
target_names=[x for x in class_names_list.keys()]
y_label_code_reduced=crea_label_vector(Data_Set)

size ridotta 439176


In [77]:
stat_ds_list=stat_ds(y_label_code_reduced)  

[ 13361   5551  17971   5193 103562   1099  29159  33986  14754   3645
   1553  18998  21455    831  11067   2140   7937   5680 241052   3408
  11588   3878   1120  21222   1296   8306   9384    170    508  55102
   7877  93065  13457   2873  11469   4773   1228   2510     83   2395
   8326      4    390   1686  17135   1928    866    690  14836    329
   9850   2929   2186    722  21357  11237  29883  49077 107149]
esempi = 0 : 0
['3.0%', '1.3%', '4.1%', '1.2%', '23.6%', '0.3%', '6.6%', '7.7%', '3.4%', '0.8%', '0.4%', '4.3%', '4.9%', '0.2%', '2.5%', '0.5%', '1.8%', '1.3%', '54.9%', '0.8%', '2.6%', '0.9%', '0.3%', '4.8%', '0.3%', '1.9%', '2.1%', '0.0%', '0.1%', '12.5%', '1.8%', '21.2%', '3.1%', '0.7%', '2.6%', '1.1%', '0.3%', '0.6%', '0.0%', '0.5%', '1.9%', '0.0%', '0.1%', '0.4%', '3.9%', '0.4%', '0.2%', '0.2%', '3.4%', '0.1%', '2.2%', '0.7%', '0.5%', '0.2%', '4.9%', '2.6%', '6.8%', '11.2%', '24.4%']


In [81]:
Data_Set=Data_Set.reset_index(drop=True)
Data_Set

In [82]:
with open(src+'GCP_TEXT_AND_LABEL_PROCESSED_59.csv', 'w') as temp:
    Data_Set.to_csv(temp, index=False)

In [2]:
for i in range (1,127):
    print(i)

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
